## Parallel Machine Scheduling Problem

In [1]:
import numpy as np
import pandas as pd
import os
import networkx as nx
import random
from IPython.display import display
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

from pyomo.environ import *

In [ ]:
############
### Sets ###
############

# Jobs
N = {1:{'P':3,'D':5},
     2:{'P':3,'D':3},
     3:{'P':4,'D':7},
     4:{'P':1,'D':4},
     5:{'P':6,'D':6},
     6:{'P':3,'D':10},}

# Machines
M = {1:{'Name':'M1'},
     2:{'Name':'M2'},
     3:{'Name':'M3'}}

# Hierarchy between jobs: (i,j) means
# job i must be completed before j can start
# regardless of which machines they are
# assigned to
N_prec = [(1,3),(2,4),(5,6)]

In [ ]:
model = ConcreteModel()

# Define sets
model.Jobs     = Set(initialize=N.keys())
model.Machines = Set(initialize=M.keys())
model.C_max    = Set(initialize=[0])

# Define parameters
model.P = Param(model.Jobs, initialize={k:v['P'] 
       for k,v in N.items()}, within=Any)
model.D = Param(model.Jobs, initialize={k:v['D'] 
       for k,v in N.items()}, within=Any)

# Define decision variables
model.t     = Var(model.Jobs, within=NonNegativeReals)
model.c     = Var(model.Jobs, within=NonNegativeReals)
model.c_max = Var(model.C_max, within=NonNegativeReals)
model.y     = Var(model.Jobs,model.Jobs, within=Binary)
model.x     = Var(model.Jobs,model.Machines, within=Binary)

In [ ]:
# Define objective function
model.obj = Objective(expr=sum(model.c_max[c] for c in model.c_max), sense=minimize)

In [ ]:
# Define constraints
model.job_assigned_once = ConstraintList()
for i in model.Jobs:
    model.job_assigned_once.add(expr=sum(model.x[i,m] 
                                         for m in model.Machines)==1)
    
model.completion_time = ConstraintList()
for i in model.Jobs:
    model.completion_time.add(model.c[i] >= model.t[i]+model.P[i])
    
model.max_completion_time = ConstraintList()
for i in model.Jobs:
    model.max_completion_time.add(model.c_max[0] >= model.c[i])

bigM = 200    
model.time_precedence_1 = ConstraintList()
for i in model.Jobs:
    for j in model.Jobs:
        if j > i:
            if (i,j) not in N_prec:
                for m in model.Machines:
                    model.time_precedence_1.add(model.t[i]+model.P[i]-model.t[j] 
                                          + bigM*model.y[i,j]+bigM*model.x[i,m]+ 
                                          bigM*model.x[j,m] <= 3*bigM)


model.time_precedence_2 = ConstraintList()
for i in model.Jobs:
    for j in model.Jobs:
        if j > i:
            if (i,j) not in N_prec:
                for m in model.Machines:
                    model.time_precedence_2.add(model.t[j]+model.P[j]-model.t[i]
                                          - bigM*model.y[i,j] <= 0)


model.time_precedence_given = ConstraintList()
for i in model.Jobs:
    for j in model.Jobs:
        if j != i:
            if (i,j) in N_prec:
                for m in model.Machines:
                    model.time_precedence_given.add(model.t[i]+model.P[i]<=model.t[j])

model.upper_bound_t = ConstraintList()
for i in model.Jobs:
    model.upper_bound_t.add(model.t[i]<=model.D[i]-model.P[i])
    
model.upper_bound_c = ConstraintList()
for i in model.Jobs:
    model.upper_bound_c.add(model.c[i]<=model.D[i])

In [ ]:
model.write('PMSP.lp', io_options={'symbolic_solver_labels': True})

# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model)  

In [ ]:
# Print the results
print('')
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
print('Overall assignment cost:', model.obj())
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
print('')

print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
print('Assignment of jobs to machines:')
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
N_m = {k:[] for k in M.keys()}
for i in model.Jobs:
    for m in model.Machines:
        if model.x[(i,m)].value >= 0.99:
            print(f'Job {i} - Machine {m}')
            N_m[m].append(i)

print('')

print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
print('Start of processing time of jobs:')
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
T_i = {}
for i in model.Jobs:
    print(f'Job {i}: {model.t[i].value}')
    T_i[i] = model.t[i].value

In [ ]:
#########################
### Plotting solution ###
#########################
from matplotlib.patches import Rectangle
axis_font  = {'fontname':'Arial', 'size':'15'}

random.seed(42)
H = 1

plt.close('all')
fig, ax = plt.subplots()
for m in M.keys():
    if len(N_m[m])>0:
        
        for i in N_m[m]:
            ax.add_patch(Rectangle((T_i[i],2*H*(m-1)),N[i]['P'],H,
                 edgecolor = 'green',
                 facecolor =  [random.randint(0,255)/255, 
                               random.randint(0,255)/255, 
                               random.randint(0,255)/255 ],
                 fill=True,
                 lw=1))
            plt.text((2*T_i[i]+N[i]['P'])/2,
                     (2*(m-1)+2*(m-1)+H)/2,
                     str(i),fontsize=25,color='w')
           
ax.set_xlim(0,model.obj())
ax.set_ylim(0,2*max(M.keys()))

y_ticks_pos    = [(2*H*(m-1)+2*H*(m-1)+H)/2 for m in M.keys()]
y_ticks_labels = [str(m) for m in M.keys()]

ax.set_yticks(y_ticks_pos,labels=y_ticks_labels)
ax.set_xlabel('Time (h)',**axis_font)
ax.set_ylabel('Machine',**axis_font)
ax.grid(True)
plt.show()
fig.savefig('sol_PMSP.png', format='png', dpi=1000, bbox_inches='tight',
         transparent=True,pad_inches=0.02) 